# Pizza Menu

This notebook showcases the use of [Lynxius](https://www.lynxius.ai/) evaluators and was used to generate [Lynxius documentation](https://docs.lynxius.ai/).

In [1]:
# First, we have to setup Lynxius API key
import os
import sys
from getpass import getpass
sys.path.append("../")

if not (lynxius_api_key := os.getenv("LYNXIUS_API_KEY")):
    lynxius_api_key = getpass("🔑 Enter your Lynxius API key: ")

os.environ["LYNXIUS_API_KEY"] = lynxius_api_key
os.environ["LYNXIUS_BASE_URL"] = "https://platform.lynxius.ai"

In [2]:
# Makes it easier to iterate
%load_ext autoreload
%autoreload 2

# Running BERTScore

In [3]:
from lynxius.client import LynxiusClient
from lynxius.evals.bert_score import BertScore

client = LynxiusClient()

# add tags for frontend filtering
label = "PR #111"
tags = ["GPT-4", "chat_pizza", "summarization", "PROD", "Pizza-DB:v2"]
bert_score = BertScore(
    label=label,
    tags=tags,
    level="word",
    presence_threshold=0.55
)

bert_score.add_trace(
    # reference from Wikipedia (https://github.com/lynxius/lynxius-docs/blob/main/docs/public/images/san_marzano_wikipedia_reference.png)
    reference=(
        "The tomato sauce of Neapolitan pizza must be made with San Marzano "
        "tomatoes or pomodorini del Piennolo del Vesuvio."
    ),
    # output from OpenAI GPT-4 (https://github.com/lynxius/lynxius-docs/blob/main/docs/public/images/san_marzano_gpt4_output.png)
    output=(
        "San Marzano tomatoes are traditionally used in Neapolitan pizza sauce."
    )
)

client.evaluate(bert_score)

'974a8271-3a84-48f4-88fd-d01a3a27e609'

# Running Answer Correctness

In [7]:
from lynxius.client import LynxiusClient
from lynxius.evals.answer_correctness import AnswerCorrectness

client = LynxiusClient()

# add tags for frontend filtering
label = "PR #111"
tags = ["GPT-4", "chat_pizza", "q_answering", "PROD", "Pizza-DB:v2"]
answer_correctness = AnswerCorrectness(label=label, tags=tags)

answer_correctness.add_trace(
    query="What is pizza quattro stagioni? Keep it short.",
    # reference from Wikipedia (https://github.com/lynxius/lynxius-docs/blob/main/docs/public/images/quattro_stagioni_wikipedia_reference.png)
    reference=(
        "Pizza quattro stagioni ('four seasons pizza') is a variety of pizza "
        "in Italian cuisine that is prepared in four sections with diverse "
        "ingredients, with each section representing one season of the year. "
        "Artichokes represent spring, tomatoes or basil represent summer, "
        "mushrooms represent autumn and the ham, prosciutto or olives represent "
        "winter."
    ),
    # output from OpenAI GPT-4 (https://github.com/lynxius/lynxius-docs/blob/main/docs/public/images/quattro_stagioni_gpt4_output.png)
    output=(
        "Pizza Quattro Stagioni is an Italian pizza that represents the four "
        "seasons through its toppings, divided into four sections. Each section "
        "features ingredients typical of a particular season, like artichokes "
        "for spring, peppers for summer, mushrooms for autumn, and olives or "
        "prosciutto for winter."
    )
)

client.evaluate(answer_correctness)

'8bd7188b-ea18-4f34-b089-05cb756b821c'

# Running Semantic Similarity

In [5]:
from lynxius.client import LynxiusClient
from lynxius.evals.semantic_similarity import SemanticSimilarity

client = LynxiusClient()

# add tags for frontend filtering
label = "PR #111"
tags = ["GPT-4", "chat_pizza", "info_retrieval", "PROD", "Pizza-DB:v2"]
semantic_similarity = SemanticSimilarity(label=label, tags=tags)

semantic_similarity.add_trace(
    # reference from Wikipedia (https://github.com/lynxius/lynxius-docs/blob/main/docs/public/images/quattro_stagioni_wikipedia_reference.png)
    reference=(
        "Pizza quattro stagioni ('four seasons pizza') is a variety of pizza "
        "in Italian cuisine that is prepared in four sections with diverse "
        "ingredients, with each section representing one season of the year. "
        "Artichokes represent spring, tomatoes or basil represent summer, "
        "mushrooms represent autumn and the ham, prosciutto or olives represent "
        "winter."
    ),
    # output from OpenAI GPT-4 (https://github.com/lynxius/lynxius-docs/blob/main/docs/public/images/quattro_stagioni_gpt4_output.png)
    output=(
        "Pizza Quattro Stagioni is an Italian pizza that represents the four "
        "seasons through its toppings, divided into four sections. Each section "
        "features ingredients typical of a particular season, like artichokes "
        "for spring, peppers for summer, mushrooms for autumn, and olives or "
        "prosciutto for winter."
    )
)

client.evaluate(semantic_similarity)

'8a5662c3-4f18-40af-b337-8271df58565f'

# Running Context Precision

In [6]:
# %pip install llama-index-llms-openai
# %pip install matplotlib

In [7]:
# import os
import pandas as pd
from llama_index.core import VectorStoreIndex, Document
from llama_index.core.response.notebook_utils import display_source_node

os.environ["OPENAI_API_KEY"] = "YOUR-OPENAI-KEY"

# Read the CSV file into a DataFrame
csv_file_path = "data/pizza_wikipedia.csv"  # adjust the path as needed
df = pd.read_csv(csv_file_path)

# Convert the DataFrame into a list of Documents
documents = [Document(text=row["text"]) for _, row in df.iterrows()]

# Create the index from the documents
index = VectorStoreIndex.from_documents(documents)

# Retrieve nodes
retriever = index.as_retriever(similarity_top_k=4)
retrieved_nodes = retriever.retrieve("Which tomato sauce is used in neapolitan pizza? Keep it short.")

for node in retrieved_nodes:
    display_source_node(node, source_length=289)

**Node ID:** aecea8f0-0395-4e45-be25-82fe90f7127d<br>**Similarity:** 0.8461458079746771<br>**Text:** Pizza marinara, also known as pizza alla marinara, is a style of pizza in Neapolitan cuisine seasoned with only tomato sauce, extra virgin olive oil, oregano and garlic. It is supposedly the oldest tomato-topped pizza.<br>

**Node ID:** 339eb18c-5c2b-472b-bc24-a163db2d16bc<br>**Similarity:** 0.8450720279096852<br>**Text:** Neapolitan pizza (Italian: pizza napoletana; Neapolitan: pizza napulitana), also known as Naples-style pizza, is a style of pizza made with tomatoes and mozzarella cheese. The tomatoes must be either San Marzano tomatoes or pomodorini del Piennolo del Vesuvio, which grow on the volcani...<br>

**Node ID:** f4ff4dba-90e0-4842-a34d-01f9c211f673<br>**Similarity:** 0.7993399980232716<br>**Text:** Pizza quattro stagioni ('four seasons pizza') is a variety of pizza in Italian cuisine that is prepared in four sections with diverse ingredients, with each section representing one season of the year. Artichokes represent spring, tomatoes or basil represent summer, mushrooms represent...<br>

**Node ID:** b90b60da-9292-4318-bd70-7bbe1aac432a<br>**Similarity:** 0.7988321996271018<br>**Text:** The first pizzeria in the U.S. was opened in New York City's Little Italy in 1905. Common toppings for pizza in the United States include anchovies, ground beef, chicken, ham, mushrooms, olives, onions, peppers, pepperoni, salami, sausage, spinach, steak, and tomatoes.<br>

In [8]:
from lynxius.client import LynxiusClient
from lynxius.evals.context_precision import ContextPrecision

from lynxius.rag.types import ContextChunk

# We used LlamaIndex to rank our Wikipedia texts based on the `query` below
# reference from Wikipedia (https://github.com/lynxius/lynxius-docs/blob/main/docs/public/images/llamaindex_pizza_top4_rank.png)
context = [
    ContextChunk(
        document="Pizza marinara, also known as pizza alla marinara, is a style of pizza in Neapolitan cuisine seasoned with only tomato sauce, extra virgin olive oil, oregano and garlic. It is supposedly the oldest tomato-topped pizza.",
        relevance=0.8461458079746771
    ),
    ContextChunk(
        document="Neapolitan pizza (Italian: pizza napoletana; Neapolitan: pizza napulitana), also known as Naples-style pizza, is a style of pizza made with tomatoes and mozzarella cheese. The tomatoes must be either San Marzano tomatoes or pomodorini del Piennolo del Vesuvio, which grow on the volcanic plains to the south of Mount Vesuvius.",
        relevance=0.8450720279096852
    ),
    ContextChunk(
        document="Pizza quattro stagioni ('four seasons pizza') is a variety of pizza in Italian cuisine that is prepared in four sections with diverse ingredients, with each section representing one season of the year. Artichokes represent spring, tomatoes or basil represent summer, mushrooms represent autumn and the ham, prosciutto or olives represent winter.",
        relevance=0.7993399980232716
    ),
    ContextChunk(
        document="The first pizzeria in the U.S. was opened in New York City's Little Italy in 1905. Common toppings for pizza in the United States include anchovies, ground beef, chicken, ham, mushrooms, olives, onions, peppers, pepperoni, salami, sausage, spinach, steak, and tomatoes.",
        relevance=0.7988321996271018
    )
]

client = LynxiusClient()

# add tags for frontend filtering
label = "PR #111"
tags = ["GPT-4", "chat_pizza", "info_retrieval", "PROD", "Pizza-DB:v2"]
context_precision = ContextPrecision(label=label, tags=tags)

context_precision.add_trace(
    query="Which tomato sauce is used in neapolitan pizza? Keep it short.",
    # reference from Wikipedia (https://github.com/lynxius/lynxius-docs/blob/main/docs/public/images/san_marzano_wikipedia_reference.png)
    reference=(
        "The tomato sauce of Neapolitan pizza must be made with San Marzano "
        "tomatoes or pomodorini del Piennolo del Vesuvio."
    ),
    context=context
)

client.evaluate(context_precision)

'0b32b4f7-ace8-4c99-a2e4-e4e3e2e8b2ae'

# Running Custom Evaluator

In [2]:
# When using CustomEval make sure that the final verdict is printed at the very
# bottom of the resonse, with no other characters
IS_SPICY_PIZZA="""
You need to evaluate if a pizza description matches a given spiciness level 
and vegetarian indication. If both match, the verdict is 'correct'; otherwise, 
it's 'incorrect'. Provide a very short explanation about how you arrived to 
your verdict. The verdict must be printed at the very bottom of your response, 
on a new line, and it must not contain any extra characters.
Here is the data:
***********
Candidate answer: {output}
***********
Spiciness level: {spicy}
***********
Vegetarian indication: {vegetarian}
"""

In [3]:
from lynxius.client import LynxiusClient
from lynxius.evals.custom_eval import CustomEval

client = LynxiusClient()

# add tags for frontend filtering
label = "PR #111"
tags = ["GPT-4", "chat_pizza", "spiciness", "PROD", "Pizza-DB:v2"]
custom_eval = CustomEval(label=label, tags=tags, prompt_template=IS_SPICY_PIZZA)

custom_eval.add_trace(
    # output from OpenAI GPT-4 (https://github.com/lynxius/lynxius-docs/blob/main/docs/public/images/hawaiian_pizza_gpt4_output.png)
    values={
        "output": "Hawaiian pizza: tomato sauce, pineapple and ham.",
        "spicy": "Not spicy",
        "vegetarian": "NO"
    }
)

client.evaluate(custom_eval)

'193d12fa-4a13-49b7-8c72-b51509fd4246'

# Running JSON Diff

In [2]:
from lynxius.client import LynxiusClient
from lynxius.evals.json_diff import JsonDiff

client = LynxiusClient()

# add tags for frontend filtering
label = "PR #111"
tags = ["GPT-4", "chat_pizza", "payload_generation", "PROD", "Pizza-DB:v2"]
json_diff = JsonDiff(label=label, tags=tags)

json_diff.add_trace(
    # reference from 'NAPIZZA SF (https://github.com/lynxius/lynxius-docs/blob/main/docs/public/images/napizza_san_francisco_menu.png)
    reference={
        "margherita": 19.0,
        "pepperoni": 21.0,
        "beer": 6.0,
        "fixed_menus": [
            {
                "menu_name": "baby",
                "pizza": "margerita",
                "drink": "Coca-Cola",
                "price": 24.0,
            },
            {
                "menu_name": "adult",
                "pizza": "pepperoni",
                "drink": "beer",
                "price": 27.0,
            }
        ]
    },
    # output from PizzaMenu LLM App
    output={
        "margherita": 39.0,
        "pepperoni": 21.0,
        "beer": 6.0,
        "fixed_menus": [
            {
                "menu_name": "baby",
                "pizza": "margerita",
                "drink": "Coca-Cola",
                "price": 24.0,
            },
            {
                "menu_name": "adult",
                "pizza": "peppers",
                "drink": "beer",
                "price": 27.0,
            }
        ]
    },
    weights={
        "margherita": 1.0,  # getting the pizza wrong is bad!
        "pepperoni": 1.0,   # getting the pizza wrong is bad!
        "beer": 0.25,       # getting the beer wrong is ok
        "fixed_menus": {    # getting the menu wrong is bad! (0.3 + 0.2 + 0.5)
            "menu_name": 0.0,
            "pizza": 0.3,
            "drink": 0.2,
            "price": 0.5,     # price is the most important thing in the menu
        }
    }
)

client.evaluate(json_diff)

'9895b04e-0e07-4eab-bcea-30d425400eac'